<a href="https://colab.research.google.com/github/cramosme/LLM-Phishing-Detection/blob/main/LLM_Phishing_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone git repo, need to run once per session i tink
# !git clone https://github.com/cramosme/LLM-Phishing-Detection.git

# %cd LLM-Phishing-Detection

# !ls

Cloning into 'LLM-Phishing-Detection'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 34 (delta 11), reused 18 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (34/34), 1.16 MiB | 8.67 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/LLM-Phishing-Detection
data  LLM_Phishing_Detection.ipynb  README.md


In [2]:
import pandas as pd

# Load the CSVs
dfLLMPhish = pd.read_csv(f"/content/LLM-Phishing-Detection/data/llm-generated/phishing-fixed.csv") # Have to use Fixed version and not original because original didnt quote text correctly so parsing was messed up
print(f"Loaded {len(dfLLMPhish)} emails")
display(dfLLMPhish.info())

print("\n")

dfHumanPhish = pd.read_csv(f"/content/LLM-Phishing-Detection/data/human-generated/phishing.csv") # Issue only occured with the LLM ones, human generated we can just use the original dataset's files
print(f"Loaded {len(dfHumanPhish)} emails")
display(dfHumanPhish.info())

Loaded 1000 emails
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1000 non-null   object
 1   label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


None



Loaded 1000 emails
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sender    1000 non-null   object
 1   receiver  978 non-null    object
 2   date      1000 non-null   object
 3   subject   998 non-null    object
 4   body      1000 non-null   object
 5   urls      1000 non-null   int64 
 6   label     1000 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


None